In [ ]:
#########没有在de且IR的转录本的AS site找到RES

In [ ]:
exon_region <- read.csv("/disk1/wenqing/tmp_data/ASD/switchList_exon.csv")
head(exon_region)

In [50]:
###计算de且IR的转录本的RNA编辑水平（差异）
celltype <- 'L5_6_CC'
de_IR_iso_celltype <- read.csv(paste0("/disk1/wenqing/tmp_data/ASD/deX_isoform_IR/de_isoform_IR_",celltype,".csv"))
head(de_IR_iso_celltype)


,X,iso_ref,gene_ref,isoform_id,gene_id,condition_1,condition_2,gene_name,gene_biotype,iso_biotype,⋯,iso_log2_fold_change,iso_q_value,IF_overall,IF1,IF2,dIF,isoform_switch_q_value,gene_switch_q_value,PTC,IR
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
1,78473,isoComp_00000763,geneComp_00000304,ENST00000496418.5_3,ACAD11,ASD,Control,ACAD11,protein_coding,retained_intron,⋯,7.8519505,NA,0.14886667,0.0000000,0.2977333,0.2977333,0.0009007804,9.007804e-04,TRUE,1
2,60934,isoComp_00017790,geneComp_00004783,ENST00000528718.6_6,DGAT1,ASD,Control,DGAT1,protein_coding,protein_coding,⋯,-0.4289815,NA,0.67866667,1.0000000,0.3573333,-0.6426667,0.0003501966,8.751818e-05,FALSE,2
3,65533,isoComp_00033146,geneComp_00014124,ENST00000553714.1_3,ERO1A,ASD,Control,ERO1A,protein_coding,retained_intron,⋯,8.2753882,NA,0.09383333,0.0000000,0.1876667,0.1876667,0.0029398197,2.939820e-03,FALSE,1
4,37109,isoComp_00041640,geneComp_00016586,ENST00000476746.6_4,HSPD1,ASD,Control,HSPD1,protein_coding,retained_intron,⋯,8.2219776,NA,0.05870000,0.0000000,0.1174000,0.1174000,0.0075260574,7.526057e-03,FALSE,2
5,39477,isoComp_00084302,geneComp_00029190,ENST00000704415.1_1,TCF7L2,ASD,Control,TCF7L2,protein_coding,protein_coding_CDS_not_defined,⋯,-7.4389210,NA,0.40248333,0.8049667,0.0000000,-0.8049667,0.0018427193,1.842719e-03,FALSE,1


In [51]:
de_IR_iso_exon_region <- exon_region[exon_region$isoform_id %in% de_IR_iso_celltype$isoform_id,]
head(de_IR_iso_exon_region)

,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
154857,154857,chr2,198351310,198351922,613,-,ENST00000476746.6_4,HSPD1,HSPD1
154876,154876,chr2,198352582,198352760,179,-,ENST00000476746.6_4,HSPD1,HSPD1
154895,154895,chr2,198353041,198353215,175,-,ENST00000476746.6_4,HSPD1,HSPD1
154914,154914,chr2,198353726,198353971,246,-,ENST00000476746.6_4,HSPD1,HSPD1
154934,154934,chr2,198354921,198355020,100,-,ENST00000476746.6_4,HSPD1,HSPD1
154953,154953,chr2,198358048,198358216,169,-,ENST00000476746.6_4,HSPD1,HSPD1


In [52]:
#拓宽转录本exon边界上下游5bp寻找RNA编辑位点
de_IR_iso_exon_region$start <- de_IR_iso_exon_region$start-5
de_IR_iso_exon_region$end <- de_IR_iso_exon_region$end+5


In [53]:

new_de_IR_iso <- de_IR_iso_exon_region[,c(2,3,4,7,8)]
head(new_de_IR_iso)
write.table(new_de_IR_iso,paste0("/disk1/wenqing/tmp_data/ASD/deX_isoform_IR/de_isoform_IR_",celltype,".newExd.txt"),quote=F,col.names=F,row.names=F)

,seqnames,start,end,isoform_id,gene_id
,<chr>,<dbl>,<dbl>,<chr>,<chr>
154857,chr2,198351305,198351927,ENST00000476746.6_4,HSPD1
154876,chr2,198352577,198352765,ENST00000476746.6_4,HSPD1
154895,chr2,198353036,198353220,ENST00000476746.6_4,HSPD1
154914,chr2,198353721,198353976,ENST00000476746.6_4,HSPD1
154934,chr2,198354916,198355025,ENST00000476746.6_4,HSPD1
154953,chr2,198358043,198358221,ENST00000476746.6_4,HSPD1


In [35]:
#res是否落在可变剪接的供受体位点上
#构建供受体位点上下游5bp的潜在res位点
donor_sites_1 <- paste0(de_IR_iso_exon_region$seqnames,"_",as.character(as.integer(de_IR_iso_exon_region$end)+1))

acceptor_sites_2 <- paste0(de_IR_iso_exon_region$seqnames,"_",as.character(as.integer(de_IR_iso_exon_region$start)-2))

all_possible_sites <- c(donor_sites_1,acceptor_sites_2)

donor_sites_info <- rep("donor_ds_1_G",times=length(de_IR_iso_exon_region$seqnames))
acceptor_sites_info <- rep("acceptor_us_2_A",times=length(de_IR_iso_exon_region$seqnames))

all_sites_info <- c(donor_sites_info,acceptor_sites_info)
all_sites <- as.data.frame(cbind(all_possible_sites,all_sites_info))


In [36]:
head(all_sites)

,all_possible_sites,all_sites_info
,<chr>,<chr>
1,chr1_93913863,donor_ds_1_G
2,chr1_93965141,donor_ds_1_G
3,chr1_93987693,donor_ds_1_G
4,chr1_93989049,donor_ds_1_G
5,chr1_93989881,donor_ds_1_G
6,chr1_93995314,donor_ds_1_G


In [37]:
##读取编辑位点
asd_only_resInAS <- read.table(paste0("/disk1/wenqing/tmp_data/ASD/resInSpliceSites_WzGeneInfo/asd_only/union_resInSpliceSites_",celltype,".WzGeneInfo.txt"))
#dim(asd_only_resInAS)

ctr_only_resInAS <- read.table(paste0("/disk1/wenqing/tmp_data/ASD/resInSpliceSites_WzGeneInfo/ctr_only/union_resInSpliceSites_",celltype,".WzGeneInfo.txt"))


In [38]:
intersect(asd_only_resInAS$V15,all_sites$all_possible_sites)
intersect(ctr_only_resInAS$V15,all_sites$all_possible_sites)

character(0)

character(0)